**Ingesting all landing zone parquet files to bronze delta files**

In [0]:
%sql
SHOW EXTERNAL LOCATIONS;

In [0]:
#Creating ingest function and adding ingestion timestamp to each table
from pyspark.sql.functions import current_timestamp

def ingest_bronze_and_save(file_path, table_name, schema_name):
    print(f'Reading File: {table_name}')

    df = spark.read.format('parquet').load(file_path)

    print(f'Transforming: {table_name}')
    df = df.withColumn('bronze_ingestion_timestamp', current_timestamp())

    print(f'Saving: {table_name}')
    df.write.mode('overwrite').option('overwriteSchema','True').format('delta').saveAsTable(f"adlslmcompany_bronze.{schema_name}.{table_name}")

    print(f"Success: {schema_name}.{table_name}")

In [0]:
%run "./Bronze_Config"  

In [0]:
df = spark.read.format('parquet').load('abfss://landingzone@adlslmcompany.dfs.core.windows.net/SalesLT/SalesLT_Address.parquet')

In [0]:

from concurrent.futures import ThreadPoolExecutor, as_completed


# Processing items
def process_config(item):
    file_path = configs[item]['file_path']
    schema_name = configs[item]['schema_name']
    table_name = configs[item]['table_name']
    ingest_bronze_and_save(file_path=file_path, table_name=table_name, schema_name=schema_name)

# Max workers
max_workers = 25

# Executing and managing threads
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(process_config, item): item for item in configs}

    # Results by item
    for future in as_completed(futures):
        item = futures[future]
        try:
            future.result()
            print(f"{item} succeed.")
        except Exception as e:
            print(f"Error {item}: {e}")
